In [1]:
import gmsh
import sys
import math

# Before using any functions in the Python API, Gmsh must be initialized:
gmsh.initialize()

gmsh.clear()
# Next we add a new model named "t1" (if gmsh.model.add() is not called a new
# unnamed model will be created on the fly, if necessary):
gmsh.model.add("HEU-MET-FAST-003=1")


Info    : Clearing all models and views...
Info    : Done clearing all models and views


In [56]:
# Define geometry parameters (example: two spheres)
inner_radius = 6.7820
outer_radius = 11.8620

In [58]:
inner_sphere = gmsh.model.occ.addSphere(0, 0, 0, inner_radius)
outer_sphere = gmsh.model.occ.addSphere(0, 0, 0, outer_radius)

In [59]:
cutResult = gmsh.model.occ.cut([(3, outer_sphere)], [(3, inner_sphere)], removeTool=False)
# Synchronize the CAD kernel with the Gmsh model
gmsh.model.occ.synchronize()
# Identify the volumes: we expect two volumes (the inner sphere and the shell)
volEntities = gmsh.model.getEntities(3)
print("Volumes:", volEntities)
shell_volume = None
for (dim, tag) in volEntities:
    if tag != inner_sphere:
        shell_volume = tag
        break
    if shell_volume is None:
        raise RuntimeError("Could not identify the shell volume.")
# Define physical groups for the inner sphere and shell

Volumes: [(3, 1), (3, 2), (3, 3), (3, 4)]                                                                                 


In [60]:
# Define physical groups for the inner sphere and shell
gmsh.model.addPhysicalGroup(3, [inner_sphere], tag=1)
gmsh.model.setPhysicalName(3, 1, "Inner")
gmsh.model.addPhysicalGroup(3, [shell_volume], tag=2)
gmsh.model.setPhysicalName(3, 2, "Shell")
# Create a MathEval field to control the mesh size based on z.
fieldTag = gmsh.model.mesh.field.add("MathEval")
# gmsh.model.mesh.field.setString(fieldTag, "F", "if(z-1e-4<0,0.9,2.3)")
gmsh.model.mesh.field.setString(fieldTag, "F", "0.95")
# Use this field as the background mesh field.
gmsh.model.mesh.field.setAsBackgroundMesh(fieldTag)

In [61]:
# Generate the 3D mesh
gmsh.model.mesh.generate(3)
# Save mesh in VTK format
gmsh.write("two_spheres.vtk")
gmsh.write("two_spheres.msh")
if "-nopopup" not in sys.argv:
    gmsh.fltk.run()
    gmsh.finalize()

Info    : Meshing 1D...
Info    : [ 10%] Meshing curve 2 (Circle)
Info    : [ 40%] Meshing curve 5 (Circle)
Info    : [ 60%] Meshing curve 8 (Circle)
Info    : [ 90%] Meshing curve 11 (Circle)
Info    : Done meshing 1D (Wall 0.000648075s, CPU 0.001178s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Sphere, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 2 (Sphere, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 3 (Sphere, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 4 (Sphere, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.265187s, CPU 0.254921s)
Info    : Meshing 3D...
Info    : 3D Meshing 4 volumes with 3 connected components
Info    : Tetrahedrizing 834 nodes...
Info    : Done tetrahedrizing 842 nodes (Wall 0.00757408s, CPU 0.007486s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.015182s, CPU 0.0153s)
Info    : Found